# Import Packages
To run this code, make sure to install the following imported packages. The package DAL is the written package from this repository

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import torch
import torchvision
import torch.optim as optim

import numpy as np

from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

import DAL

# Check Device
Having a gpu to run this code is not a requirement, but will help the tests run substantially faster!

In [2]:
try:
    device = torch.device('cuda:0')
except:
    device = torch.device('cpu')

# Create Data-sets

We use tensorflow datasets, specifically their CIFAR-10 Data-set. This is a good baseline to use for the algorithm. 

In [3]:
batch_size = int(1e2)

image, label = tfds.as_numpy(tfds.load(
    'cifar10',
    split='train', 
    batch_size=-1, 
    as_supervised=True,
))

train_images = torch.Tensor(image[:,:,:,:]).transpose(2,3).transpose(1,2)
train_labels = torch.Tensor(label)

image, label = tfds.as_numpy(tfds.load(
    'cifar10',
    split='test', 
    batch_size=-1, 
    as_supervised=True,
))

test_images = torch.Tensor(image[:,:,:,:]).transpose(2,3).transpose(1,2)
test_labels = torch.Tensor(label)

test_dataset = TensorDataset(test_images, test_labels)
test_loader  = DataLoader(test_dataset, batch_size=batch_size)

# Set up model
Pytorch's Resnet-18 is used for classification, with SGD for optimization. Cross-Entropy is the loss function 

In [4]:
net    = torchvision.models.resnet18(pretrained=False).to(device)
net.fc = nn.Linear(512,10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

# Set up vectors and run experiment 
Here, we set up the vectors capture to record the data. The experiments are simulated here as well. 

In [5]:
numIters  = 20
ADMMIters = 1000

num_examples_increase_list = [10,15,20,25,30,35,40,45,50]

accuracy_array_optimize = np.zeros([len(num_examples_increase_list),numIters,2])
accuracy_array_random   = np.zeros([len(num_examples_increase_list),numIters,2])

image_variance_optim = np.zeros([len(num_examples_increase_list),numIters])
image_variance_rand  = np.zeros([len(num_examples_increase_list),numIters])

alpha_variance_optim = np.zeros([len(num_examples_increase_list),numIters])
alpha_variance_rand  = np.zeros([len(num_examples_increase_list),numIters])


In [6]:
for ii in range(len(num_examples_increase_list)):
    
    iterations            = 1
    num_examples_increase = num_examples_increase_list[ii]
    
    for iter_num in range(20):

        network = DAL.Node_Network(10,train_images,train_labels,10)
        data_selector = DAL.Data_Optimizer(network,ADMMIters)

        random_list = DAL.sample_random_points(network,iterations,num_examples_increase)

        hub_images = network.hub.images
        hub_labels = network.hub.labels

        net.apply(DAL.reset_weights)

        DAL.train_model(net,hub_images,hub_labels,test_loader,criterion,optimizer)

        loss_array,accuracy_array,optim_alpha,optim_img = DAL.run_test(net,data_selector,test_loader,criterion,optimizer,iterations,num_examples_increase,network,hub_images,hub_labels)

        accuracy_array_optimize[ii,iter_num,:] = accuracy_array

        hub_images = network.hub.images
        hub_labels = network.hub.labels

        loss_array,accuracy_array,rand_alpha,rand_img = DAL.run_test_random(net,test_loader,criterion,optimizer,iterations,num_examples_increase,network,hub_images,hub_labels,random_list)

        accuracy_array_random[ii,iter_num,:]   = accuracy_array
        
        print("Optimization Image Variance: \t" + str(optim_img.var().item()))
        print("Random Image Variance: \t\t" + str(rand_img.var().item()))
        print("Optimization Alpha Variance: \t" + str(optim_alpha.var().item()))
        print("Random Alpha Variance: \t\t" + str(rand_alpha.var().item()))

        image_variance_optim[ii,iter_num] = optim_img.mean().item()
        image_variance_rand[ii,iter_num]  = rand_img.mean().item()
        alpha_variance_optim[ii,iter_num] = optim_alpha.var(0).mean().item()
        alpha_variance_rand[ii,iter_num]  = rand_alpha.var(0).mean().item()



5090
---------------------------------Optimization Output Final---------------------------------


Loss: 0.03
Accuracy: 37.45000005722046


5090
---------------------------------Random Output Final---------------------------------


Loss: 0.03
Accuracy: 37.560000038146974


Optimization Image Variance: 	417651.90625
Random Image Variance: 		421522.65625
Optimization Alpha Variance: 	2.59552001953125
Random Alpha Variance: 		2.6023361682891846
5090
---------------------------------Optimization Output Final---------------------------------


Loss: 0.03
Accuracy: 37.60000003814697


5090
---------------------------------Random Output Final---------------------------------


Loss: 0.03
Accuracy: 37.90000005722046


Optimization Image Variance: 	423115.625
Random Image Variance: 		424572.28125
Optimization Alpha Variance: 	4.108326435089111
Random Alpha Variance: 		4.10872745513916
5090
---------------------------------Optimization Output Final---------------------------------


Loss: 0.03


5090
---------------------------------Optimization Output Final---------------------------------


Loss: 0.03
Accuracy: 37.740000076293946


5090
---------------------------------Random Output Final---------------------------------


Loss: 0.03
Accuracy: 37.40000007629394


Optimization Image Variance: 	386038.59375
Random Image Variance: 		389768.375
Optimization Alpha Variance: 	4.99738073348999
Random Alpha Variance: 		5.023597240447998
5135
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 38.15000003814697


5135
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 37.66000009536743


Optimization Image Variance: 	425687.0625
Random Image Variance: 		412247.875
Optimization Alpha Variance: 	5.062289237976074
Random Alpha Variance: 		4.956180095672607
5135
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Acc

5135
---------------------------------Optimization Output Final---------------------------------


Loss: 0.03
Accuracy: 37.62000007629395


5135
---------------------------------Random Output Final---------------------------------


Loss: 0.03
Accuracy: 37.15000005722046


Optimization Image Variance: 	460841.90625
Random Image Variance: 		433823.1875
Optimization Alpha Variance: 	2.3000502586364746
Random Alpha Variance: 		2.2618727684020996
5135
---------------------------------Optimization Output Final---------------------------------


Loss: 0.03
Accuracy: 38.03000003814697


5135
---------------------------------Random Output Final---------------------------------


Loss: 0.03
Accuracy: 37.750000076293944


Optimization Image Variance: 	402496.15625
Random Image Variance: 		405022.4375
Optimization Alpha Variance: 	6.343503475189209
Random Alpha Variance: 		6.3525238037109375
5180
---------------------------------Optimization Output Final---------------------------------


Loss: 0

5180
---------------------------------Optimization Output Final---------------------------------


Loss: 0.03
Accuracy: 36.680000171661376


5180
---------------------------------Random Output Final---------------------------------


Loss: 0.03
Accuracy: 36.91000020980835


Optimization Image Variance: 	462680.4375
Random Image Variance: 		431907.40625
Optimization Alpha Variance: 	9.013672828674316
Random Alpha Variance: 		8.73203182220459
5180
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 36.79000005722046


5180
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 36.090000038146975


Optimization Image Variance: 	490337.03125
Random Image Variance: 		426808.53125
Optimization Alpha Variance: 	5.62730073928833
Random Alpha Variance: 		5.42233943939209
5180
---------------------------------Optimization Output Final---------------------------------


Loss: 0.03


5225
---------------------------------Optimization Output Final---------------------------------


Loss: 0.03
Accuracy: 38.480000019073486


5225
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 38.3


Optimization Image Variance: 	468524.875
Random Image Variance: 		456005.34375
Optimization Alpha Variance: 	2.955005407333374
Random Alpha Variance: 		2.8886263370513916
5225
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 37.15000011444092


5225
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 37.390000076293944


Optimization Image Variance: 	394844.03125
Random Image Variance: 		403817.5625
Optimization Alpha Variance: 	3.298238515853882
Random Alpha Variance: 		3.2936627864837646
5225
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 

5270
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 38.07000003814697


5270
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 36.79000009536743


Optimization Image Variance: 	401656.375
Random Image Variance: 		390901.3125
Optimization Alpha Variance: 	5.663719654083252
Random Alpha Variance: 		5.525716781616211
5270
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 36.79000015258789


5270
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 36.32000009536743


Optimization Image Variance: 	439838.65625
Random Image Variance: 		433843.03125
Optimization Alpha Variance: 	7.707590103149414
Random Alpha Variance: 		7.663088798522949
5270
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
A

5315
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 37.910000038146975


5315
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 38.07000005722046


Optimization Image Variance: 	410279.9375
Random Image Variance: 		421190.40625
Optimization Alpha Variance: 	9.426626205444336
Random Alpha Variance: 		9.435070037841797
5315
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 38.410000038146975


5315
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 38.82000003814697


Optimization Image Variance: 	394219.96875
Random Image Variance: 		392986.46875
Optimization Alpha Variance: 	5.510727405548096
Random Alpha Variance: 		5.49849796295166
5315
---------------------------------Optimization Output Final---------------------------------


Loss: 0.0

5360
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 38.47000001907349


5360
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 38.52000007629395


Optimization Image Variance: 	418609.28125
Random Image Variance: 		406755.625
Optimization Alpha Variance: 	8.84917163848877
Random Alpha Variance: 		8.831124305725098
5360
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 37.0500001335144


5360
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 37.090000038146975


Optimization Image Variance: 	400206.0
Random Image Variance: 		387238.875
Optimization Alpha Variance: 	11.812902450561523
Random Alpha Variance: 		11.55076789855957
5360
---------------------------------Optimization Output Final---------------------------------


Loss: 0.03
Accura

5405
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 39.39000005722046


5405
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 39.37000003814697


Optimization Image Variance: 	432644.75
Random Image Variance: 		434704.90625
Optimization Alpha Variance: 	3.526592493057251
Random Alpha Variance: 		3.517238140106201
5405
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 38.25000003814697


5405
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 39.53000003814697


Optimization Image Variance: 	458401.71875
Random Image Variance: 		440437.40625
Optimization Alpha Variance: 	4.140748500823975
Random Alpha Variance: 		4.004537105560303
5405
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
A

5450
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 36.43000007629394


5450
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 36.16000011444092


Optimization Image Variance: 	458700.1875
Random Image Variance: 		400861.65625
Optimization Alpha Variance: 	12.534464836120605
Random Alpha Variance: 		11.648999214172363
5450
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 38.27


5450
---------------------------------Random Output Final---------------------------------


Loss: 0.02
Accuracy: 37.55000007629395


Optimization Image Variance: 	394791.125
Random Image Variance: 		381263.28125
Optimization Alpha Variance: 	3.592923164367676
Random Alpha Variance: 		3.554729700088501
5450
---------------------------------Optimization Output Final---------------------------------


Loss: 0.02
Accuracy: 3

# Save Experiments

In [7]:
for ii in range(9):
    np.savetxt("new_ADMM_optimization_values_" + str(num_examples_increase_list[ii]) + ".csv", accuracy_array_optimize[ii,:,:], delimiter=",")
    np.savetxt("new_ADMM_random_values_" + str(num_examples_increase_list[ii]) + ".csv", accuracy_array_random[ii,:,:], delimiter=",")
    
for ii in range(9):
    np.savetxt("new_ADMM_optimization_alpha_values_" + str(num_examples_increase_list[ii]) + ".csv", alpha_variance_optim[ii,:], delimiter=",")
    np.savetxt("new_ADMM_random_alpha_values_" + str(num_examples_increase_list[ii]) + ".csv", alpha_variance_rand[ii,:], delimiter=",")
    
for ii in range(9):
    np.savetxt("new_ADMM_optimization_img_values_" + str(num_examples_increase_list[ii]) + ".csv", image_variance_optim[ii,:], delimiter=",")
    np.savetxt("new_ADMM_random_img_values_" + str(num_examples_increase_list[ii]) + ".csv", image_variance_rand[ii,:], delimiter=",")